In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!pip install peft
!pip install trl
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
# !pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


In [2]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
# from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = '42dot/42dot_LLM-SFT-1.3B'

# model.bin 파일의 경로
new_model = "/content/gdrive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Test_10epoch_Qlora"

In [6]:
pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [7]:
pip install accelerate --upgrade

In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model)


model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/5.76G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/870k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using device: cuda


In [5]:
import torch


# 모델을 CPU로 이동
model = model.to('cuda:0')

# 모델의 파라미터를 FP32로 변경
# model = model.float()

# 텍스트 입력
text = "철근 콘크리트의 장점은 무엇인가요?"

# 토큰화
tokens = tokenizer(text, return_tensors="pt")
tokens = {key: val.to('cuda:0') for key, val in tokens.items()}


# 텍스트 생성 설정
max_length = 400  # 생성할 텍스트의 최대 길이
num_return_sequences = 1  # 생성할 텍스트 시퀀스의 수

# 텍스트 생성
with torch.no_grad():
    generated_ids = model.generate(
        tokens["input_ids"],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(f"Generated text: {generated_text}")

Generated text: 철근 콘크리트의 장점은 무엇인가요?


In [6]:
model = model.merge_and_unload()

AttributeError: 'LlamaForCausalLM' object has no attribute 'merge_and_unload'

In [7]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:

import warnings

# 모든 경고 메시지를 무시하도록 설정합니다.
warnings.filterwarnings("ignore")

# 기존에 학습된 모델을 불러옵니다. 여기서 AutoModelForCausalLM은 인과적 언어 모델을 위한 클래스입니다.
# 'model_name'은 불러올 모델의 이름을 지정합니다.
# 'low_cpu_mem_usage'는 CPU 메모리 사용을 최소화하도록 설정합니다.
# 'return_dict'는 모델의 출력을 딕셔너리 형태로 반환하도록 설정합니다.
# 'torch_dtype=torch.float16'는 모델 연산에서 사용할 데이터 타입을 16비트 부동소수점으로 설정합니다.
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)

# PeftModel 클래스를 이용하여 기존 모델에 새로운 모델을 추가합니다.
model = PeftModel.from_pretrained(base_model, new_model)

# 모델의 리소스를 정리합니다.
model = model.merge_and_unload()

# 토크나이저를 로드합니다. 이는 텍스트를 모델이 이해할 수 있는 형태로 변환하는데 사용됩니다.
# 'trust_remote_code=True'는 원격 코드를 신뢰하고 사용하도록 설정합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 토크나이저의 패딩 토큰을 문장 종료 토큰으로 설정합니다.
tokenizer.pad_token = tokenizer.eos_token

# 패딩이 오른쪽에 적용되도록 설정합니다.
tokenizer.padding_side = "right"

# 텍스트 입력을 정의합니다. 사용자에게 정보를 요청하는 프롬프트입니다.
prompt = "도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 건축 전문가적 입장에서 500자 이내로 요약해서 답변해주세요."

# 주어진 텍스트를 토큰화합니다.
tokens = tokenizer(text=f"{prompt}\n답변:", return_tensors="pt")

# 모델을 GPU로 옮깁니다.
model = model.to('cuda:0')

# 토큰화된 데이터를 GPU로 옮깁니다.
tokens = tokens.to('cuda:0')


# 텍스트를 생성합니다. 이는 모델에 입력 토큰을 넣고, 지정된 설정에 따라 텍스트를 생성합니다.
generated_ids = model.generate(
    tokens["input_ids"],           # 모델에 입력할 토큰 ID들입니다. 이전에 토큰화한 결과가 여기에 사용됩니다.
    max_length=600,         # 생성할 텍스트의 최대 길이를 지정합니다. 이 경우 최대 350개의 토큰까지 생성될 수 있습니다.
    min_length=50,                 # 생성할 텍스트의 최소 길이를 지정합니다. 적어도 50개의 토큰은 생성될 것입니다.
    num_return_sequences=1,  # 생성할 텍스트 시퀀스의 수를 지정합니다. 여기서는 1개의 시퀀스만 생성하도록 설정했습니다.
    pad_token_id=tokenizer.eos_token_id,  # 패딩에 사용할 토큰 ID를 지정합니다. 여기서는 문장 종료 토큰(eos_token)의 ID를 사용합니다.
    eos_token_id=tokenizer.eos_token_id,  # 문장 종료 토큰의 ID를 지정합니다. 생성된 텍스트가 이 토큰을 만나면 생성을 중단합니다.
    no_repeat_ngram_size=3,      # 반복되는 n-gram의 크기를 지정합니다. 여기서는 3개의 연속된 토큰이 반복되지 않도록 설정합니다.
    temperature=0.3,                  # 생성 과정에서의 무작위성 정도를 지정합니다. 0으로 설정하면 가장 확률이 높은 토큰만 선택되어 매우 결정론적인 결과가 생성됩니다.
    # top_p=0.6,    #0과 1 사이의 값을 설정합니다. 0에 가까울수록 가장 높은 확률을 가진 토큰만 선택되어 반복 가능성이 높아지고, 1에 가까울수록 다양한 토큰이 선택되어 반복 가능성이 낮아집니다.
    # repetition_penalty=4.8 #0 이상의 값을 설정합니다. 값이 높을수록 동일한 단어 또는 토큰이 연속적으로 나타나는 것을 더욱 강하게 방지합니다.
)


# 생성된 텍스트를 디코딩합니다. 이는 생성된 토큰 ID를 실제 텍스트로 변환합니다.
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# 원하는 부분만 추출합니다.
start_token = "답변:"
start_idx = generated_text.find(start_token)
if start_idx != -1:
    # "답변:" 이후의 텍스트만 추출합니다.
    final_text = generated_text[start_idx + len(start_token):].strip()
else:
    # "답변:" 토큰이 없는 경우, 전체 텍스트를 사용합니다.
    final_text = generated_text

# 최종적으로 추출된 텍스트를 출력합니다.
print(final_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


도배지에 발생하는 녹은 자국 주된 원인 중 하나는 도배지 사이에 틈이 있는 경우입니다. 이러한 틈으로 인해 도배지가 흡수되는 수분이 외부로 배출되지 못하고 내부에 갇히게 되어 녹이 발생할 수 있습니다.
해결 방법으로는 도배지의 틈을 최소화하고, 도배지를 고정시키는 데 사용되는 접착제를 더 효과적으로 사용하는 것이 있습니다. 또한, 도배를 할 때 도배지와 벽 사이의 공간을 충분히 확보하여 수분이 배출될 수 있는 통로를 만들어주는 것이 중요합니다.
또한, 도포되는 접착제의 양을 적절히 조절하여 도배지에서 발생하는 녹을 방지할 수 있습니다


In [10]:
#랭체인 부분

import pandas as pd
train = pd.read_csv("/content/gdrive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Data/train.csv", encoding='utf-8')

from itertools import product

# 빈 리스트 생성
train_data = []

# 질문과 답변의 조합을 생성
for q, a in product([f"질문_{x}" for x in range(1, 3)], [f"답변_{x}" for x in range(1, 6)]):
    for i in range(len(train)):
        # 값이 문자열이 아닌 경우를 고려하여 str() 함수를 사용
        question = str(train.at[i, q])
        answer = str(train.at[i, a])
        train_data.append("질문: " + question + " 답변 : " + answer)

pd.DataFrame(train_data).to_csv("train_data1.csv",index=False,encoding='utf-8')

In [11]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='train_data1.csv',encoding='utf-8')
data = loader.load()

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

modelPath = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [13]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(data, embedding=embeddings)
db.save_local("faiss_index")
retriever = db.as_retriever(search_kwargs={"k": 4})

In [14]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline

# model_name = '42dot/42dot_LLM-SFT-1.3B'

# model.bin 파일의 경로
# new_model = "/content/gdrive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/5epoch_Qlora"

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,device=0) # 기존값: device=0
hf = HuggingFacePipeline(pipeline=pipe)

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """From an architectural expert's perspective, please review the context below and respond to the question strictly within 500 characters. Ensure that your response is in Korean and do not repeat the same words or phrases. Only the answer to the provided question is required.
Context: {context}

Question: {question}

Answer:
"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | hf
    | StrOutputParser()
)

In [16]:
for chunk in rag_chain.stream("도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 건축 전문가적 입장에서 300자 이내로 요약해서 답변해주세요."):
    print(chunk, end="", flush=True)

도배지에 녹이 발생하는 주요 원인은 다음과 같습니다. 1. 높은 습도: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 발생할 수 있습니다. 2. 누수: 누수로 인해 도배지가 젖어 있는 상태가 지속되면 곰팡이와 녹이 발생할 수 있습니다. 3. 결로 현상: 결로 현상으로 인해 도배지 상에 얼룩이 생길 수 있으며, 이는 녹이 발생할 수 있는 원인이 될 수 있습니다. 4. 기타 요인: 기타 요인으로 인해 도배지 상에 녹이 발생할 수 있으며, 예를 들어, 화재로 인한 열 손상, 화학 물질에 의한 오염 등이 있습니다. 이러한 원인에 따라 도배지에 녹이 발생하는 것을 방지하기 위해 다음과 같은 해결 방법을 고려할 수 있습니다. 1. 제습기 가동: 높은 습도를 관리하기 위해 제습기를 사용하여 실내의 습도를 조절하는 것이 중요합니다. 2. 환기: 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 필요합니다. 3. 보수 작업: 누수 문제를 해결하기 위해 보수 작업을 수행하여 누수로 인한 곰팡이와 녹을 예방할 수 있습니다. 4. 전문가의 도움: 도배지에 녹이 발생하는 문제를 해결하기 위해 전문가의 도움을 받는 것이 좋습니다. 전문가의 지식과 기술을 활용하여 도배지 상의 녹을 예방하고 복구하는 작업을 수행할 수 있습니다.

In [17]:
chunk

'도배지에 녹이 발생하는 주요 원인은 다음과 같습니다. 1. 높은 습도: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 발생할 수 있습니다. 2. 누수: 누수로 인해 도배지가 젖어 있는 상태가 지속되면 곰팡이와 녹이 발생할 수 있습니다. 3. 결로 현상: 결로 현상으로 인해 도배지 상에 얼룩이 생길 수 있으며, 이는 녹이 발생할 수 있는 원인이 될 수 있습니다. 4. 기타 요인: 기타 요인으로 인해 도배지 상에 녹이 발생할 수 있으며, 예를 들어, 화재로 인한 열 손상, 화학 물질에 의한 오염 등이 있습니다. 이러한 원인에 따라 도배지에 녹이 발생하는 것을 방지하기 위해 다음과 같은 해결 방법을 고려할 수 있습니다. 1. 제습기 가동: 높은 습도를 관리하기 위해 제습기를 사용하여 실내의 습도를 조절하는 것이 중요합니다. 2. 환기: 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 필요합니다. 3. 보수 작업: 누수 문제를 해결하기 위해 보수 작업을 수행하여 누수로 인한 곰팡이와 녹을 예방할 수 있습니다. 4. 전문가의 도움: 도배지에 녹이 발생하는 문제를 해결하기 위해 전문가의 도움을 받는 것이 좋습니다. 전문가의 지식과 기술을 활용하여 도배지 상의 녹을 예방하고 복구하는 작업을 수행할 수 있습니다.'

In [ ]:
chunk

In [18]:
import pandas as pd
test = pd.read_csv("/content/gdrive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Data/test.csv")

In [19]:
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [20]:
from tqdm import tqdm

result = []

for i in tqdm(range(len(test))):
  _id = test.at[i,'id']
  _q = test.at[i,'질문']
  _a = []
  for chunk in rag_chain.stream(_q):
      _a.append(chunk)
      print(chunk, end="", flush=True)
  result.append(
      {
          "id":_id,
          "대답":" ".join(_a)
      }
  )
  print()

  0%|          | 0/130 [00:00<?, ?it/s]


방청 페인트의 종류에는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각각의 특성과 용도에 맞게 사용됩니다.

원목사이딩을 사용하는 것에는 몇 가지 단점이 있을 수 있습니다. 첫째, 원목사이딩은 내구성이 약할 수 있습니다. 따라서 시간이 지나면서 갈라지거나 손상될 수 있습니다. 둘째, 원목사이딩은 습기에 취약할 수 있습니다. 습기가 침투하면 사이딩이 부식될 수 있습니다. 셋째, 원목사이딩은 단열 효과가 낮을 수 있습니다. 사이딩이 사이딩 사이를 밀폐하지 못하면 열이 빠져나갈 수 있습니다.

따라서 방청 페인트를 선택할 때는 원목사이딩의 단점을 고려하여 적절한 방청 페인트를 선택하는 것이 중요합니다. 또한, 사이딩을 사용할 때에도 적절한 시공 방법을 사용하여 사이딩이 손상되지 않도록 주의해야 합니다.

  1%|          | 1/130 [00:08<18:19,  8.53s/it]



1. 공간 내 높은 습도 원인 : 높은 습도로 인해 도배지 안쪽의 금속이 녹이 발생할 수 있습니다. 책임 소재는 건물의 소유자나 거주자가 있습니다. 해결 방법은 제습기를 가동하거나 환기를 통해 공간 내 적정 습도를 유지하는 것입니다.

2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임 소재는 건물의 소유자나 거주자가 있습니다. 해결 방법은 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것입니다.

3. 녹 오염의 원인 : 도배지에 녹이 묻어나오는 현상은 공간 내 습도가 높거나 누수가 발생하여 발생할 수 있습니다. 책임 소재는 건물의 소유자나 거주자가 있습니다. 해결 방법은 제습기를 가동하거나 환기를 통해 공간 내 적정 습도를 유지하는 것입니다. 또한, 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요합니다.

4. 녹 오염의 해결 방법 : 도배지에 녹이 묻어나온 경우, 먼저 제습기를 가동하거나 환기를 통해 공간 내 적정한 습도를 유지하는 것이 중요합니다. 또한, 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요합니다. 이러한 보수작업은 전문가의 도움을 받아 진행하는 것이 좋습니다.

5. 녹 오염의 주된 원인은 무엇인가요?

- 공간 내 높은 습도 : 높은 습도로 인해 도배지 안쪽의 금속이 녹이 발생할 수 있습니다.
- 누수 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다.
- 습도 : 공간 내 적정 습도를 유지하는 것이 중요합니다.
- 누수 : 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요합니다.

이러한 원인들이 도배지에 녹이 묻어나오는 현상을 발생시키는 주된 원인입니다.

  2%|▏         | 2/130 [00:22<25:08, 11.78s/it]

...

 98%|█████████▊| 128/130 [21:37<00:17,  8.73s/it]


석고수정이 발생하는 주요 원인은 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있기 때문입니다. 이러한 원인으로 인해 석고수정이 발생하는 경우, 해결 방법은 다음과 같습니다.

1. 석고보드의 손상이나 변형을 확인하기 위해 정밀한 진단을 받는 것이 중요합니다.
2. 석고보드의 결함을 해결하기 위해 전문가의 도움을 받는 것이 좋습니다.
3. 석고보드의 결함을 해결하기 위해 석고보드를 부분적으로 잘라내고 보강하는 방법을 사용할 수 있습니다.
4. 석고보드의 결함을 해결하기 위해 석고보드를 교체하는 방법을 사용할 수 있습니다.

이러한 방법들은 석고수정을 해결하는 데에 효과적입니다. 하지만, 석고보드의 결함을 해결하기 위해 반점을 바인더나 수성 프라이머로 코팅하는 방법은 반점이 다시 생기는 문제가 발생할 수 있으므로, 가능한 한 석고보드를 부분적으로 잘라내고 보강하는 방법을 사용하는 것이 좋습니다.

 99%|█████████▉| 129/130 [21:44<00:08,  8.22s/it]



카페트의 기대 수명은 일반적으로 6년 정도를 예상할 수 있습니다. 그러나 다다미나 가구를 자주 옮기는 공간에서 사용하는 경우에는 수명이 단축될 수 있으니 이 점을 유의하셔야 합니다. 또한, 카페트를 장시간 밟고 사용하거나 주변 환경이 먼지나 습기로 인해 카페트에 손상을 줄 수 있는 경우에는 수명이 단축될 수 있습니다.

오리지널징크의 장점에는 다음과 같은 것들이 있습니다. 첫째, 오리지널징크는 내구성이 뛰어나며 오랜 기간 동안 사용할 수 있습니다. 둘째, 오리지널징크는 자연스러운 패턴과 색상을 가지고 있어 인테리어에 조화로운 효과를 줄 수 있습니다. 셋째, 오리지널징크는 내화성이 뛰어나 화재 발생 시에도 안전하게 사용할 수 있습니다.

따라서, 카페트의 기대 수명을 연장하기 위해서는 정기적인 청소와 유지보수를 통해 카페트의 수명을 연장하는 것이 중요합니다. 또한, 오리지널징크의 장점을 활용하여 인테리어에 조화로운 효과를 주고 화재 발생 시에도 안전하게 사용할 수 있는 카페트를 선택하는 것도 좋은 방법입니다.

100%|██████████| 130/130 [21:52<00:00, 10.09s/it]

In [21]:
import pickle

with open("result.pkl",'wb') as f:
  pickle.dump(result,f)

In [22]:
from sentence_transformers import SentenceTransformer
_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

for i in range(len(result)):
  result[i]['embedding'] = _model.encode(result[i]['대답'].replace("\u200b"," "))

In [23]:
submission = []

for i in range(len(result)):
  tmp = {"id":result[i]['id'],}
  for j in range(len(result[i]['embedding'])):
    tmp[f'vec_{j}'] = result[i]['embedding'][j]
  submission.append(
      tmp
  )

In [24]:
pd.DataFrame(submission).to_csv("/content/gdrive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Submission/42dot_RAG__Test_10epochs_500.csv",encoding='utf-8',index=False)